<a href="https://colab.research.google.com/github/its-Kumar/Deep_Learning/blob/master/NN_from_scratch/NueralNetwork_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent and Nueral Network

Implement Gradient Descent for Nueral Nwetwork and implement a Neural Network from scratch and compare with keras NN. 

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
tf.__version__

'2.3.0'

## Loading the dataset

In [2]:
df = pd.read_csv('insurance_data.csv')
df

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


## Splitting into train and test set

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df[['age', 'affordibility']],
    df.bought_insurance,
    test_size=0.2,
    random_state=0
)

In [4]:
len(X_train)

22

## Scaling the data

In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [6]:
X_train_scaled

,age,affordibility
24,0.50,1
13,0.29,0
20,0.21,1
25,0.54,1
16,0.25,0
1,0.25,0
10,0.18,1
27,0.46,1
26,0.23,1
8,0.62,1


## Build a Model with keras

In [7]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,),
                       activation='sigmoid',
                       kernel_initializer='ones',
                       bias_initializer='zeros')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              )
model.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 2ms/step - loss: 0.7960 - accuracy: 0.4091
Epoch 2/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7955 - accuracy: 0.4091
Epoch 3/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7950 - accuracy: 0.4091
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7944 - accuracy: 0.4091
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7939 - accuracy: 0.4091
Epoch 6/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7934 - accuracy: 0.4091
Epoch 7/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7929 - accuracy: 0.4091
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7923 - accuracy: 0.4091
Epoch 9/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7918 - accuracy: 0.4091
Epoch 10/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7913 - accuracy: 0.4091
Epoch 11/500
1/1 [===========

In [8]:
X_test_scaled

,age,affordibility
2,0.47,1
22,0.40,1
14,0.49,1
17,0.58,1
5,0.56,1
11,0.28,1


### Keras model prediction

In [9]:
y_pred = model.predict(X_test_scaled)
y_pred

array([[0.6250502 ],
       [0.6140306 ],
       [0.62817556],
       [0.6421043 ],
       [0.6390288 ],
       [0.59487075]], dtype=float32)

In [51]:
coef, intercept = model.get_weights()
coef, intercept

(array([[0.6679039],
        [0.6164324]], dtype=float32), array([-0.41930738], dtype=float32))

## Build Model from scratch without using tensorflow

### sigmoid function

In [11]:
def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))

In [12]:
sigmoid(10)

0.9999546021312976

### prediction function

In [13]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [14]:
prediction_function(.47, 1)

0.62505020865451

### log loss function

In [29]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

### sigmoid using numpy

In [16]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

### Gradient Descent Implementation

In [30]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias


In [31]:
gradient_descent(X_train_scaled['age'],
                 X_train_scaled['affordibility'],
                 y_train,
                 1000,
                 .4)


Epoch:0, w1:0.9644915668168719, w2:0.9224250019701856, bias:-0.1521744437835525, loss:0.7960362503355486
Epoch:1, w1:0.9366743363654499, w2:0.8570969777186894, bias:-0.2838815971644586, loss:0.7394293670708099
Epoch:2, w1:0.9160190930945575, w2:0.8035200242324965, bias:-0.39666489545168215, loss:0.6980698848391689
Epoch:3, w1:0.9017701861063158, w2:0.7606695089288332, bias:-0.4926120466423785, loss:0.6686193819406211
Epoch:4, w1:0.8930731543176171, w2:0.7272344982483193, bias:-0.5740228037875161, loss:0.6479960845015643
Epoch:5, w1:0.8890759364765383, w2:0.7018265973050505, bias:-0.6431543649176596, loss:0.6336578618954344
Epoch:6, w1:0.8889918350364353, w2:0.6831195783238424, bias:-0.7020686534913516, loss:0.6236667598398875
Epoch:7, w1:0.8921292648124783, w2:0.6699231996615576, bias:-0.7525646139800298, loss:0.6166236529551378
Epoch:8, w1:0.8978991061148434, w2:0.6612102292494013, bias:-0.796166156177148, loss:0.6115548071590623
Epoch:9, w1:0.9058095436860382, w2:0.6561158934541762, 

(8.276057309257109, 1.3538527130157803, -4.496619150713256)

## Building our Model 

### Defining the class

In [44]:
class MyNeuralNetwork():

    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold):
        """
        Parameters:
        X: features data
        y: labeled data
        """
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],
                                                            X['affordibility'],
                                                            y,
                                                            epochs,
                                                            loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")

    def predict(self, X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

        
    def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold):
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = self.w1 * age + self.w2 * affordability + self.bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            self.w1 = self.w1 - rate * w1d
            self.w2 = self.w2 - rate * w2d
            self.bias = self.bias - rate * bias_d

            if i%5==0 or (i==epochs-1):
                print(f'Epoch:{i}, w1:{self.w1}, w2:{self.w2}, bias:{self.bias}, loss:{loss}')

            if loss<=loss_thresold:
                break

        return self.w1, self.w2, self.bias


### Model

In [49]:
custom_model = MyNeuralNetwork()
custom_model.fit(X_train_scaled, y_train, epochs=1000, loss_thresold=0.46)

Epoch:0, w1:0.9644915668168719, w2:0.9224250019701856, bias:-0.1521744437835525, loss:0.7960362503355486
Epoch:5, w1:0.8890759364765383, w2:0.7018265973050505, bias:-0.6431543649176596, loss:0.6336578618954344
Epoch:10, w1:0.9154552200900856, w2:0.6539236724735189, bias:-0.8675276612678249, loss:0.6049128343543392
Epoch:15, w1:0.9800574396529999, w2:0.6693652761532601, bias:-0.9903692468815464, loss:0.5961440387682162
Epoch:20, w1:1.057780818233721, w2:0.7037401912418658, bias:-1.0758774642779125, loss:0.5901630074558274
Epoch:25, w1:1.1398544034718647, w2:0.7418049456763838, bias:-1.146436376852304, loss:0.5848330839234127
Epoch:30, w1:1.2232120309833507, w2:0.7785915222202571, bias:-1.209926753055886, loss:0.5798540151893508
Epoch:35, w1:1.3067540202230254, w2:0.8126490680852346, bias:-1.2692388740409404, loss:0.5751525327590069
Epoch:40, w1:1.3900639770355059, w2:0.8437133541856828, bias:-1.3255260401538713, loss:0.5706894193199624
Epoch:45, w1:1.4729666356566584, w2:0.8719051856492

### our model prediction

In [52]:
coef, intercept

(array([[0.6679039],
        [0.6164324]], dtype=float32), array([-0.41930738], dtype=float32))

In [55]:
y_pred = custom_model.predict(X_test_scaled)
y_pred

2     0.617881
22    0.539849
14    0.639278
17    0.728045
5     0.709521
11    0.403655
dtype: float64

In [57]:
model.predict(X_test_scaled)

array([[0.6250502 ],
       [0.6140306 ],
       [0.62817556],
       [0.6421043 ],
       [0.6390288 ],
       [0.59487075]], dtype=float32)

In [56]:
y_test

2     1
22    1
14    1
17    1
5     1
11    0
Name: bought_insurance, dtype: int64